In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

0) Classroom'da paylaşılacak Lebron.csv dosyasını bilgisayarınıza indirin. Dönem sonunda da kullanacağız. Yükledikten sonra kaggle'a gidin ve new dataset sekmesinde 'Lebron' olarak isimlendirerek yükleyin. Sonrasında yeni notebook yarattığınızda, orada your datasets kısmından ekleyebilir olacaksınız.


In [ ]:
Lebron = pd.read_csv("../input/lebron/train.csv", index_col=0)
Lebron

1) Sadece Lebron James'in şutlarını gösteren bir dataframe oluşturun.


In [ ]:
Lebron.loc[Lebron['shoot player'] == 'LeBron James']

2) Verilerimizin içerisinde her sütün için kaç tane dolu veri var, percentile'ları kaç, ortalama değerleri kaç gibi değişkenleri gösteren kodu yazınız. Bütün kolonların isimlerini yazdırın.


In [ ]:
datatypes =['object', 'float', 'int']

Lebron.describe( include = datatypes)

3) Sonrasında açılan notebook'tan verileri okutun. İlk 5 entry'sini yazdırın. Gözünüze çarpan şeyleri comment olarak ekleyin.


In [ ]:
Lebron.head()
# verilerin NaN değerleri içeriyor

4) Kaç sütun ve kaç satır içerdiğini yazdırın.


In [ ]:
satir = len(Lebron)
sutun = len(Lebron.columns)
print("satir:",satir,"sutun:",sutun)

5) Değişken tiplerimiz neler? (Float,int,str) gibi, bunları yazdırın.


In [ ]:
Lebron.dtypes


6) Cleveland takımında en yüksek yüzdeyle şut atan oyuncu kimdir? Bunu bulurken, bütün şutları değerlendirin, serbest atış, 2 sayılık atış ve 3 sayılık atış olması önemli değil. Bu değer kaçtır?


In [ ]:
Clevand_team_shoots = Lebron.loc[ ((Lebron['home team'] == 'CLE') & (Lebron['home game'] == 'Yes')) | 
                                 ((Lebron['away team'] == 'CLE') & (Lebron['home game'] == 'No')) ]
CLE_home_team = Clevand_team_shoots['home team'].map(lambda desc: "CLE" in desc).sum()
CLE_away_team = Clevand_team_shoots['away team'].map(lambda desc: "CLE" in desc).sum()
CLE_Shoot_Number = CLE_home_team + CLE_away_team
players = Clevand_team_shoots['shoot player'].value_counts()
player = (players / CLE_Shoot_Number)*100

print(" Cleveland takımında en yüksek yüzdeyle şut atan :",player.iloc[0:1])

7) Cleveland takımındaki her oyuncunun attığı toplam şut sayısını bulunuz.


In [ ]:
CLE_players = Lebron.loc[ ((Lebron['home team'] == 'CLE') & (Lebron['home game'] == 'Yes')) | 
                         ((Lebron['away team'] == 'CLE') & (Lebron['home game'] == 'No')) ]
players = CLE_players['shoot player'].value_counts()
players

8) Lebron James'in attığı şutların içerisinde maksimum x değerine sahip entry'nin indexini bulunuz.


In [ ]:
Lebron_james_shoots = Lebron.loc[Lebron['shoot player'] == 'LeBron James' ]
Lebron_james_shoots_max = Lebron_james_shoots.loc[Lebron_james_shoots['location x'] == Lebron_james_shoots['location x'].max() ]
Lebron_james_shoots_max



9) Bir fonksiyon tanımlayın. Buradaki amaç location y, ve location x verilerini fonksiyona verdiğinizde, şutun atıldığı noktanın potaya kuşbakışı uzaklığını(feet) buldurmaktır. Bu uzaklıkları 0-10,10-20,20-30, 30+ olacak şekilde, yakın, orta, uzak ve orta sahadan gibi isimlerle sınıflandırın. Bu sınıflandırmayı verinize yeni bir sütun(feature) olarak ekleyin.


In [ ]:
import math


Max_y = Lebron['location y'].max()
Min_y = Lebron['location y'].min()
Max_x = Lebron['location x'].max()
Min_x = Lebron['location x'].min()
#  1) 3 inch = 0.25 feet ----    2) pota = 63 = 21 * 3    ----    3) 21 * 0.25 =  5.25  -----     4)  5.25*10 = 52.5
Left_basketball_hoop_x = 52.5
right_basketball_hoop_x = Max_x - 52.5
basketball_hoop_y = Max_y/2
def basketball(df):
    if df['location x'] < (Max_x/2):
        return math.hypot((abs(df['location x'] - Left_basketball_hoop_x))/10, (abs(df['location y'] - basketball_hoop_y))/10 )
    else:
        return math.hypot((abs(right_basketball_hoop_x - df['location x']))/10,(abs(df['location y'] - basketball_hoop_y))/10  )

basketball_hoop_distance = Lebron.apply(basketball, axis='columns')
Lebron['distance'] = basketball_hoop_distance

def feet_shots(Lebron):
    if Lebron['distance'] < 10 :
        return 'near'
    elif (Lebron['distance'] > 10 and Lebron['distance'] < 20):
        return 'mid'
    elif (Lebron['distance'] > 20 and Lebron['distance'] < 30):
        return 'long'
    else:
        return 'middlecourt'

feet_shots_classification = Lebron.apply(feet_shots, axis='columns')
Lebron['classification'] = feet_shots_classification
Lebron


10) Şut tiplerini bir liste haline getirin ve verilerin tümünü kullanarak her şuttan ne kadar kullanıldığını bulun. Listeyi alfabe sırasında sıralayın ve listenin sonuna kendi isimlendirdiğiniz bir şutu ekleyin. Sonra eklediğiniz bu şutu silin. Bank shot içeren indexleri bir liste haline getirin.
yanliş

In [ ]:
shoots = Lebron.groupby(['shot type']).size()
Shoots_types = pd.Series(shoots)
Shoots_types = Shoots_types.sort_index()
New_Shoots_name = pd.Series([10], index=['Z Ali shot'])
Shoots_types = Shoots_types.add(New_Shoots_name, fill_value=0)
Shoots_types

In [ ]:
Shoots_types = Shoots_types.drop(labels=['Z Ali shot'])
Shoots_types

In [ ]:
Shoots_types.loc[Shoots_types.index.str.find('Bank Shot') > -1]

11) Boş bir seri yaratın. Şut tiplerinden 'a' harfi içerenleri listede tutmak için bir kod yazın.


In [ ]:
Shoots_a = Lebron.loc[Lebron['shot type'].str.find('a') > -1] 
Shoots_colum = Shoots_a['shot type']
Shoots_serisi = pd.Series(Shoots_colum)
Shoots_serisi


12) Derste işlediğimiz, 'List comprehension' kullanarak, atılan tüm şutlardan kaç tanesinin y değerinin 250'den büyük olduğunu veren fonksiyonu yazın.
Örnek: def......
Return .....
Örnekteki gibi 2 satırda yapmanız gereklidir.

In [ ]:
def Biger_y(rows):
    return sum([row > 250 for row in rows])
    
location_y = Biger_y(Lebron['location y'])
location_y

13) Bir dictionary oluşturun. O dictionary'e verimizin quarter bilgisini gönderceğiz. O gönderdiğimiz sayıya göre ilk çeyrek, ikinci çeyrek, üçüncü çeyrek ve son çeyrek olarak değiştirsin.


In [ ]:
dictionary  = ['ilk çeyrek', 'ikinci çeyrek', 'üçüncü çeyrek' ,'son çeyrek' ]
def quarter_changer(Lebron):
    if Lebron.quarter == 1:
        return dictionary[0]
    elif Lebron.quarter == 2:
        return dictionary[1]
    elif Lebron.quarter == 3:
        return dictionary[2]
    else:
        return dictionary[3]
Lebron_with_dictionary = Lebron.apply(quarter_changer, axis='columns')
Lebron_with_dictionary

14) Bugüne kadar kullandığımız bütün module/library'leri yazın. Module'un veya library'nin hangi fonksiyonları içerdiğini yazdırın. O fonksiyonlardan görmediğimiz bir tanesini seçin, ve fonksiyonun hangi değişkenleri aldığını, o değişkenlerin anlamlarını yazdırın.


-sklearn.linear_model.TweedieRegressor
power : float, default=0
*Güç, aşağıdaki tabloya göre temel hedef dağılımını belirler.
alpha : float, default=1
*Ceza süresini çoğaltan ve böylece düzenlileştirme gücünü belirleyen sabit.
*fit_intercept : bool, default=True
 Doğrusal tahminciye bir sabitin eklenmesi gerekip gerekmediğini belirtir.
*link :  {‘auto’, ‘identity’, ‘log’}, default=’auto’
 GLM'nin bağlantı işlevi, yani doğrusal tahmin ediyor.
*max_iter : int, default=100
 Çözücü için maksimum yineleme sayısı.
*tol : float, default=1e-4
 Durdurma kriteri. lbfgs çözücü için, max{|g_j|, j = 1, ..., d} <= tol olduğunda yineleme duracaktır
*warm_startbool, default=False
True olarak ayarlanırsa, önceki çağrının çözümünü, coef_ ve intercept_ için başlatma olarak sığacak şekilde yeniden kullanın.
*verboseint, default=0
 Lbfgs çözücüsü için ayrıntılı, ayrıntı için herhangi bir pozitif sayıya ayarlayın.



numpy.savez_compressed(file, *args, **kwds)
Birkaç diziyi sıkıştırılmış .npz formatında tek bir dosyaya kaydetme
*file  :  str or file
Verilerin kaydedileceği dosya adı (dize) veya açık bir dosya (dosya benzeri nesne).
*args : Arguments, optional
Dosyaya kaydedilecek diziler. Dizilere isim atamak için lütfen anahtar kelime argümanlarını kullanın (aşağıdaki kwds'ye bakın).
*kwds:  Keyword arguments, optional
Dosyaya kaydedilecek diziler. Her dizi, karşılık gelen anahtar kelime adıyla çıktı dosyasına kaydedilecektir.


- pandas.read_pickle(filepath_or_buffer, compression='infer', storage_options=None)
* filepath_or_buffer :  str, path object, or file-like object
dize, nesnenin lokasyon (path) veya dosya benzeri nesne.
* compression :  str or dict, default ‘infer’
Diskteki verilerin anında açılması için. ardından şu uzantılardan tespit edin: ".gz", ".bz2", ".zip", ".xz" veya ".zst"
*storage_options  :   dict, optional
Belirli bir depolama bağlantısı için anlamlı olan ekstra seçenekler, ör. ana bilgisayar, bağlantı noktası, kullanıcı adı, şifre vb.



matplotlib.image.imread(fname, format=None)
fname : str or file-like
Okunacak görüntü dosyası: bir dosya adı, bir URL veya ikili okuma modunda açılan dosya benzeri bir nesne.
format  : str, optional
Verileri okumak için görüntü dosyası formatı. "png"  ,"png"  veya URL


math.fabs(x)¶
Return the absolute value of x.


15) Lebron James için Atlanta (ATL) ve New York Knicks(NYK maçlarında), kaç şut atıp, kaç şut kaçırdığını gösteren bir dataframe oluşturun. Ve onu yazdırın.
Örnek:
 Missed Scored
NYK 10 20
ATL 30 40
gibi.


In [ ]:
Lebron_james = Lebron.loc[Lebron['shoot player'] == 'LeBron James']

ATL_Team = Lebron_james.loc[ (Lebron_james['home team'] == 'ATL')|(Lebron_james['away team'] == 'ATL') ]
NYK_Team = Lebron_james.loc[ (Lebron_james['away team'] == 'NYK')|(Lebron_james['home team'] == 'NYK') ]

ATL_MISSED = ATL_Team['current shot outcome'].map(lambda desc: "MISSED" in desc).sum()
NYK_MISSED = NYK_Team['current shot outcome'].map(lambda desc: "MISSED" in desc).sum()
ATL_SCORED = ATL_Team['current shot outcome'].map(lambda desc: "SCORED" in desc).sum()
NYK_SCORED = NYK_Team['current shot outcome'].map(lambda desc: "SCORED" in desc).sum()

NYK_and_ATL = pd.DataFrame({'MISSED': [NYK_MISSED, ATL_MISSED], 'SCORED': [NYK_SCORED, ATL_SCORED]},index=['NYK', 'ATL'])
NYK_and_ATL

16) Veriyi oyuncu ismi(shoot player) ve çeyrek(quarter)'lara göre gruplandırın. Oluşturduğunuz uzaklık sütununu kullanarak, o çeyreklerde attığı maksimum uzaklıkların indexlerini buldurunuz.
Örnek:
(Shoot player)	   (quarter) 	(index)
Lebron James 		1	 5
Lebron James 		2	 23 
Not: Sütün isimleri farklı olabilir, bir önemi yok.

In [ ]:
shoots_quarter = Lebron.groupby(['shoot player','quarter']).distance.max()
shoots_quarter

17) Cleveland'lı oyuncuları ve karşılarında oynadıkları takımları gruplayın. Her clevelandlı oyuncunun rakibe attığı toplam şutun miktarına göre büyükten küçüğe sıralayın.
Örnek:

Lebron James NYK 150
Lebron James DEN 20
gibi.


In [ ]:
CLETEAM_Players_home_team = Lebron.loc[ ((Lebron['home team'] == 'CLE') & (Lebron['home game'] == 'Yes')) ]
CLETEAM_Players_away_team = Lebron.loc[ ((Lebron['away team'] == 'CLE') & (Lebron['home game'] == 'No' )) ]

CLETEAM_home_team = CLETEAM_Players_home_team.groupby(['shoot player','away team']).size()
CLETEAM_away_team = CLETEAM_Players_away_team.groupby(['shoot player','home team']).size()

CLETEAM = pd.concat([CLETEAM_home_team, CLETEAM_away_team])
CLETEAM = CLETEAM.sort_values(ascending=False)
CLETEAM

18) Sütunların data type'leri eğer float'sa integer'a, integer ise float'a çevirin. (Orijinal data üstünde yapmayın). Yazdırın.


In [ ]:
df = Lebron.copy()
df = df.dropna(axis=0)
df.points = df.points.astype('float64')
df.quarter = df.quarter.astype('float64')
df['location x'] = df['location x'].astype('int64')
df['location y'] = df['location y'].astype('int64')
df['time from last shot'] = df['time from last shot'].astype('int64')
df['points_gained'] = df['points_gained'].astype('int64')
df['difference'] = df['difference'].astype('int64')
df

19) Self Previous Shot Sütununda boş veriler var mı? Bu boş veriler doldurulmalı mı? Eğer doldurulmalıysa, fillna() fonksiyonunu kullanarak doldurun.


In [ ]:
Lebron[pd.isnull(Lebron['self previous shot'])]


In [ ]:
Lebron['self previous shot'] = Lebron['self previous shot'].fillna("Unknown")
Lebron[pd.isnull(Lebron['self previous shot'])]


20) Bütün sütun isimlerini internette sözlük kullanarak türkçeye çevirin, çevirdiğinizi gösterin.


In [ ]:
df = Lebron.copy()
df = df.rename(columns={'self previous shot':'kendi önceki çekimi',
  'player position': 'oyuncu pozisyonu',
  'home game':  'Ev oyunu',
  'location x':'x konumu',
  'opponent previous shot':'rakip önceki atış',
  'home team':'ev sahibi takım',
  'shottype': 'çekim tipi',
  'points': 'puan',
  'away team':'deplasman takımı',
  'location y':'y konumu',
  'time': 'zaman',
  'date':'tarih',
  'shoot player':'oyuncu vur',
  'time from last shot':'son atıştan itibaren geçen süre',
  'quarter':'çeyrek',
  'current shot outcome':'mevcut atış sonucu',
  'My_Team':'Benim takımım',
  'HomeAway':'Evden uzak',
  'points_gained':'points_gained',
  'difference':'fark'})
df

21) Elimizde ekstra maçlar olduğunu varsayalım. Hangi durumlarda concat, hangi durumlarda join kullanmamız gerektiğini düzgün bir şekilde açıklayın.


Bu durumlarda 
join :
Ortak bir index sahip farklı DataFrame nesnelerini birleştirmenize olanak tanır.
Concat :
Concat öğeleri bir eksen boyunca sıkıştıracaktır.
Bu, farklı DataFrame veya Series nesnelerinde verilerimiz olduğunda ancak aynı alanlara (sütunlara) sahip olduğumuzda kullanışlıdır

Genel olarak join ve concat :
join:
- İki veya daha fazla panda DataFrames/Serisini yatay olarak birleştirmek için kullanılan işlev.
- dizin üzerinde dizin (varsayılan olarak) ve dizin üzerinde sütun(lar) için dahili olarak birleştirme kullanır.
- Çağıran DataFrame'in sütun(lar)ını veya indeksini diğer nesnelerin indeksiyle (sütunlarla değil) hizalar.
- Sağ, iç ve dış birleştirme seçenekleriyle sol birleştirme varsayılanları
Concat :
- iki veya daha fazla panda DataFrames/Serisini dikey veya yatay olarak birleştirin.
- Yalnızca eksen parametresini belirterek dizinde hizalar.
- Varsayılan olarak, iç birleştirme seçeneğiyle dış birleştirme
BU yazili

22) Test verilerimizi kullanarak, şutun girip girmediğini (scored, missed, blocked) tahmin edeceğimizi varsayarak (current shot outcome), data visualization dersinde öğrendiğimiz bilgileri kullanın. (Regression plot, lmplot, barplot ve diğerleri gibi). Hangi özellikler hedef değişkenle ilintili görünüyor, yorumlayın. (Burada yoğun çaba harcayıp, gözden hiçbir şeyi kaçırmadığınıza emin olun.)


In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import LabelEncoder
Lebron_copy = Lebron.copy()
label_encoder = LabelEncoder()
Lebron_copy['date'] = label_encoder.fit_transform(Lebron_copy['date'])
Lebron_copy['time'] = label_encoder.fit_transform(Lebron_copy['time'])
Lebron_data = Lebron_copy.dropna(axis=0)

features = ['location x','points','location y','quarter','points_gained','date','time','home game']
X = Lebron_data[features]
current_shot_outcome_copy = Lebron_data['current shot outcome']

# Split into validation and training data
train_grafik_X, val_grafik_X, train_grafik_y, val_grafik_y = train_test_split(X, current_shot_outcome_copy, random_state=1)
 

In [ ]:

import pandas as pd
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# Set the width and height of the figure
plt.figure(figsize=(10,6))

# Add title
plt.title("Average SCORED, MISSED and BLOCKED")
sns.barplot(x=val_grafik_X['date'], y=val_grafik_y)

# Add label for vertical axis
plt.ylabel("current shot outcome")

In [ ]:

sns.scatterplot(x=val_grafik_X['location x'], y=val_grafik_X['location y'])



In [ ]:


sns.scatterplot(x=val_grafik_X['location x'], y=val_grafik_X['location y'], hue=val_grafik_y)


In [ ]:
sns.lmplot(x="location x", y="location y", hue="current shot outcome", data=Lebron_data)


In [ ]:
sns.swarmplot(x=val_grafik_y,
              y=val_grafik_X['location x'])

In [ ]:

sns.jointplot(x=val_grafik_X['location x'], y=val_grafik_X['location y'], kind="kde")


23) Datamızı derste ayırdığımız gibi, train validasyon olarak ikiye ayıralım. İlintili feature seçimlerimizi de yapalım. 


In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import LabelEncoder
Lebron_copy = Lebron.copy()
label_encoder = LabelEncoder()
Lebron_copy['shoot player'] = label_encoder.fit_transform(Lebron_copy['shoot player'])
Lebron_copy['player position'] = label_encoder.fit_transform(Lebron_copy['player position'])
Lebron_copy['shot type'] = label_encoder.fit_transform(Lebron_copy['shot type'])
Lebron_copy['home game'] = label_encoder.fit_transform(Lebron_copy['home game'])
Lebron_copy['away team'] = label_encoder.fit_transform(Lebron_copy['away team'])
Lebron_copy['home team'] = label_encoder.fit_transform(Lebron_copy['home team'])
Lebron_data = Lebron_copy.dropna(axis=0)

features = ['location x','points','location y','quarter','points_gained','shoot player','player position',
            'away team','home team','shot type','home game']
X = Lebron_data[features]
current_shot_outcome_copy = label_encoder.fit_transform(Lebron_data['current shot outcome'])

# Split into validation and training data
train_X, val_X, train_y, val_y = train_test_split(X, current_shot_outcome_copy, random_state=1)




24) DecisionTree ile sonuçlar elde edin ve Mean absolute value'nüzü bulun.


In [ ]:
# Define model
Lebron_model = DecisionTreeRegressor()
# Fit model
Lebron_model.fit(train_X, train_y)

# get predicted prices on validation data
val_predictions = Lebron_model.predict(val_X)
print(mean_absolute_error(val_y, val_predictions))

25) DecisionTree'nizde maksimum yaprak nodlarını(maximum_leaf_nodes) tanımlayarak, modelinizin en iyi sonuç verdiği max_leaf_nodes'i seçin. Sonrasında ona göre bir mean absolute error value'sünü bulun. Bunu neden yaptığımızı detaylı bir şekilde açıklayın.


In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
Lebron_copy = Lebron.copy()
label_encoder = LabelEncoder()
Lebron_copy['shoot player'] = label_encoder.fit_transform(Lebron_copy['shoot player'])
Lebron_copy['player position'] = label_encoder.fit_transform(Lebron_copy['player position'])
Lebron_copy['shot type'] = label_encoder.fit_transform(Lebron_copy['shot type'])
Lebron_copy['home game'] = label_encoder.fit_transform(Lebron_copy['home game'])
Lebron_copy['away team'] = label_encoder.fit_transform(Lebron_copy['away team'])
Lebron_copy['home team'] = label_encoder.fit_transform(Lebron_copy['home team'])
Lebron_data = Lebron_copy.dropna(axis=0)

features = ['location x','points','location y','quarter','points_gained','shoot player',
            'player position','shot type','away team','home team','home game']
X = Lebron_data[features]
y = label_encoder.fit_transform(Lebron_data['current shot outcome'])

# Split into validation and training data
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)

def get_DecisionTreeRegressor(max_leaf_nodes, train_X, val_X, train_y, val_y):
    model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=1)
    model.fit(train_X, train_y)
    preds_val = model.predict(val_X)
    mae = mean_absolute_error(val_y, preds_val)
    return mae
for max_leaf_nodes in [5, 40, 50, 60, 75, 100]:
    my_DecisionTreeRegressor = get_DecisionTreeRegressor(max_leaf_nodes, train_X, val_X, train_y, val_y)
    print("Max leaf nodes: " ,max_leaf_nodes," \t\t Mean Absolute Error:" ,my_DecisionTreeRegressor)

26) Random Forest'ta max_leaf_nodes kullanılır mı? Bu soruyu cevaplarken koddan yardım alıp göstermeyi unutmayın.


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
Lebron_copy = Lebron.copy()
label_encoder = LabelEncoder()
Lebron_copy['shoot player'] = label_encoder.fit_transform(Lebron_copy['shoot player'])
Lebron_copy['player position'] = label_encoder.fit_transform(Lebron_copy['player position'])
Lebron_copy['shot type'] = label_encoder.fit_transform(Lebron_copy['shot type'])
Lebron_copy['home game'] = label_encoder.fit_transform(Lebron_copy['home game'])
Lebron_copy['away team'] = label_encoder.fit_transform(Lebron_copy['away team'])
Lebron_copy['home team'] = label_encoder.fit_transform(Lebron_copy['home team'])
Lebron_data = Lebron_copy.dropna(axis=0)

features = ['location x','points','location y','quarter','points_gained','shoot player',
            'player position','shot type','away team','home team','home game']
X = Lebron_data[features]
y = label_encoder.fit_transform(Lebron_data['current shot outcome'])

# Split into validation and training data
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)

forest_model = RandomForestRegressor(random_state=1)
forest_model.fit(train_X, train_y)
melb_preds = forest_model.predict(val_X)
val_mae = mean_absolute_error(melb_preds, val_y)
print("Validation MAE when not specifying max_leaf_nodes:",val_mae)

# Using best value for max_leaf_nodes
max_leaf_nodes_forest_model = RandomForestRegressor( max_leaf_nodes=50,random_state=1)
max_leaf_nodes_forest_model.fit(train_X, train_y)
val_predictions = max_leaf_nodes_forest_model.predict(val_X)
max_leaf_nodes_val_mae = mean_absolute_error(val_predictions, val_y)
print("Validation MAE for best value of max_leaf_nodes:",max_leaf_nodes_val_mae)



27) 24. Ve 25. Soruda yaptığımız işlemin aynısını random Forest modeli için de yapın.


In [ ]:
forest_model = RandomForestRegressor(random_state=1)
forest_model.fit(train_X, train_y)
melb_preds = forest_model.predict(val_X)
val_mae = mean_absolute_error(melb_preds, val_y)
print("Validation MAE when not specifying max_leaf_nodes:",val_mae)


def get_RandomForestRegressor(max_leaf_nodes, train_X, val_X, train_y, val_y):
    iowa_model = RandomForestRegressor(max_leaf_nodes=max_leaf_nodes, random_state=1)
    iowa_model.fit(train_X, train_y)
    val_predictions = iowa_model.predict(val_X)
    val_mae = mean_absolute_error(val_predictions, val_y)
    return val_mae
for max_leaf_nodes in [5, 40, 50, 60, 75, 80 ,90]:
    my_mae = get_RandomForestRegressor(max_leaf_nodes, train_X, val_X, train_y, val_y)
    print("Max leaf nodes: " ,max_leaf_nodes," \t\t Mean Absolute Error:" ,my_mae)
    

28) Random forest ve decision tree'nin avantajlarını ve dezavantajlarını yorumlayınız.


Random Forest:
avantajlar:
1. Random Forest Ensemble Learning tekniğini kullanır,  Verilerin alt kümesinde olabildiğince çok ağaç oluşturur ve tüm ağaçların çıktısını birleştirir.
2.  Random Forest, hem sınıflandırma hem de regresyon problemlerini çözmek için kullanılabilir.
3. Random Forest, hem kategorik hem de sürekli değişkenlerle iyi çalışır.
4. Random Forest, eksik değerleri otomatik olarak işleyebilir.
5. Özellik ölçeklendirmesi gerekmez: mesafe hesaplaması yerine kural tabanlı yaklaşımı kullandığı için özellik ölçeklendirmesi gerekmez.
6. Doğrusal olmayan parametreleri verimli bir şekilde işler
7. Random Forest, eksik değerleri otomatik olarak işleyebilir.
8. Random Forest, genellikle aykırı değerlere karşı dayanıklıdır ve bunları otomatik olarak işleyebilir.
9. Random Forest çok kararlıdır. Veri kümesine yeni bir veri noktası eklnirse genel olarak algoritma çok fazla etkilenmez, ancak tüm ağaçları etkilemesi çok zordur.
10. Random Forest, gürültüden nispeten daha az etkilenir.
Dezavantajları:
1. Karmaşıklık: çok sayıda ağaç oluşturur (karar ağacında sadece bir ağacın aksine) ve çıktılarını birleştirir. Bunu yapmak için algoritma çok daha fazla hesaplama gücü ve kaynağı gerektirir. Öte yandan karar ağacı basittir ve çok fazla hesaplama kaynağı gerektirmez.
2. Daha Uzun Eğitim Süresi: çok sayıda ağaç ürettiğinden karar,  ağaçlarına kıyasla çok daha fazla eğitim süresi gerektirir.
Decision tree:
Avantajlar:
Diğer algoritmalara kıyasla karar ağaçları, ön işleme sırasında veri hazırlığı için daha az çaba gerektirir.
Bir karar ağacı, verilerin normalleştirilmesini gerektirmez.
Bir karar ağacı, verilerin ölçeklendirilmesini de gerektirmez.
Verilerdeki eksik değerler de bir karar ağacı oluşturma sürecini önemli ölçüde etkilemez.
Karar ağacı modeli çok sezgiseldir ve hem teknik ekiplere hem de paydaşlara açıklanması kolaydır.

Dezavantaj:
Verilerdeki küçük bir değişiklik, karar ağacının yapısında büyük bir değişikliğe neden olarak kararsızlığa neden olabilir.
Bir Karar ağacı için bazen hesaplama, diğer algoritmalara kıyasla çok daha karmaşık olabilir.
Karar ağacı genellikle modeli eğitmek için daha fazla zaman gerektirir.
Karar ağacı eğitimi, karmaşıklığı ve aldığı zaman daha fazla olduğu için nispeten pahalıdır.
Karar Ağacı algoritması, regresyon uygulamak ve sürekli değerleri tahmin etmek için yetersizdir.
